In [1]:
import pandas as pd
import numpy as np
import pickle
import ast
import matplotlib.pyplot as plt
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

import os
import re
import gensim
from gensim import corpora
import nltk
nltk.download
from nltk.corpus import stopwords

import requests
from gensim.utils import simple_preprocess
from pprint import pprint
from nltk.stem import PorterStemmer
from gensim.models.coherencemodel import CoherenceModel
from sklearn.model_selection import ParameterGrid
from tqdm import tqdm

from gensim.models import LdaModel
from itertools import product

import warnings
warnings.filterwarnings("ignore")

In [2]:
import joblib
# Load a scikit-learn model from a .pkl file
load_path = "/home/saji/DSprojects/BookRecommendation/model/lda_model_final.joblib"
lda_model = joblib.load(load_path)

In [22]:
dataset = pd.read_csv('/home/saji/DSprojects/BookRecommendation/data/cleaned_books.csv')

In [32]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143505 entries, 0 to 143504
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   Title          143505 non-null  object
 1   image          136593 non-null  object
 2   authors        141755 non-null  object
 3   categories     137888 non-null  object
 4   publisher      125147 non-null  object
 5   description    143505 non-null  object
 6   topic_vectors  143390 non-null  object
dtypes: object(7)
memory usage: 7.7+ MB


In [23]:
dataset.head()

,Title,image,authors,categories,publisher,description
0,Dr. Seuss: American Icon,http://books.google.com/books/content?id=IjvHQ...,['Philip Nel'],['Biography & Autobiography'],A&C Black,Philip Nel takes a fascinating look into the k...
1,Wonderful Worship in Smaller Churches,http://books.google.com/books/content?id=2tsDA...,['David R. Ray'],['Religion'],NaN,This resource includes twelve principles in un...
2,Whispers of the Wicked Saints,http://books.google.com/books/content?id=aRSIg...,['Veronica Haddon'],['Fiction'],iUniverse,Julia Thomas finds her life spinning out of co...
3,The Church of Christ: A Biblical Ecclesiology ...,http://books.google.com/books/content?id=kVqRa...,['Everett Ferguson'],['Religion'],Wm. B. Eerdmans Publishing,In The Church of Christ: A Biblical Ecclesiolo...
4,Saint Hyacinth of Poland,http://books.google.com/books/content?id=lmLqA...,['Mary Fabyan Windeatt'],['Biography & Autobiography'],Tan Books & Pub,The story for children 10 and up of St. Hyacin...


In [24]:
with open('/home/saji/DSprojects/BookRecommendation/models/corpus.pkl', 'rb') as file:
    corpus = pickle.load(file)

In [25]:
with open('/home/saji/DSprojects/BookRecommendation/models/id2word.pkl', 'rb') as file:
    id2word = pickle.load(file)

In [26]:
topic_dist = pd.DataFrame(columns=['topic distribution'])

In [33]:
# Function to get topic distribution for each document
def get_topic_distribution(lda_model, id2word, corpus):
    topic_distributions = []
    for doc_bow in corpus:
        topic_distribution = lda_model.get_document_topics(doc_bow)
        topic_distributions.append(dict(topic_distribution))
    return topic_distributions

# Create a new column 'topic_distribution' in the dataset
topic_dist['topic distribution'] = get_topic_distribution(lda_model, id2word, corpus)

# Display the dataset with the new 'topic_distribution' column
print(topic_dist['topic distribution'])

0         {1: 0.10596203, 4: 0.1509699, 5: 0.14428952, 8...
1         {7: 0.35825828, 17: 0.31998673, 28: 0.04853301...
2         {3: 0.24076997, 6: 0.031575207, 8: 0.012265164...
3         {0: 0.042262796, 2: 0.023289798, 7: 0.13346055...
4         {3: 0.18298261, 6: 0.09266154, 14: 0.045818202...
                                ...                        
143385    {14: 0.09415097, 19: 0.36684817, 20: 0.0898175...
143386    {3: 0.2745605, 9: 0.07911556, 20: 0.29505408, ...
143387    {2: 0.019166967, 3: 0.019099459, 5: 0.09926778...
143388    {2: 0.048119135, 4: 0.1280643, 5: 0.055934463,...
143389    {0: 0.21955623, 7: 0.22485073, 15: 0.29403645,...
Name: topic distribution, Length: 143390, dtype: object


In [47]:
#Topic Matrix
from sklearn.metrics.pairwise import cosine_similarity

# Assuming lda_model is your trained LDA model
num_topics = lda_model.num_topics

# Function to vectorize topic distributions
def vectorize_topic_distribution(topic_distribution, num_topics):
    if isinstance(topic_distribution,dict):
        topic_vector = np.zeros(num_topics)
        for topic, probability in topic_distribution.items():
            if topic < num_topics:  # Check if the topic index is within bounds
                topic_vector[topic] = probability
        return topic_vector
    else: 
        return np.zeros(num_topics)
    


dataset['topic_vectors'] = topic_dist['topic distribution'].apply(lambda x: vectorize_topic_distribution(x,num_topics))



In [39]:
dataset.head()

,Title,image,authors,categories,publisher,description,topic_vectors
0,Dr. Seuss: American Icon,http://books.google.com/books/content?id=IjvHQ...,['Philip Nel'],['Biography & Autobiography'],A&C Black,Philip Nel takes a fascinating look into the k...,"[0.0, 0.10596203058958054, 0.0, 0.0, 0.1509698..."
1,Wonderful Worship in Smaller Churches,http://books.google.com/books/content?id=2tsDA...,['David R. Ray'],['Religion'],NaN,This resource includes twelve principles in un...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.35825827..."
2,Whispers of the Wicked Saints,http://books.google.com/books/content?id=aRSIg...,['Veronica Haddon'],['Fiction'],iUniverse,Julia Thomas finds her life spinning out of co...,"[0.0, 0.0, 0.0, 0.24076996743679047, 0.0, 0.0,..."
3,The Church of Christ: A Biblical Ecclesiology ...,http://books.google.com/books/content?id=kVqRa...,['Everett Ferguson'],['Religion'],Wm. B. Eerdmans Publishing,In The Church of Christ: A Biblical Ecclesiolo...,"[0.04226279631257057, 0.0, 0.02328979782760143..."
4,Saint Hyacinth of Poland,http://books.google.com/books/content?id=lmLqA...,['Mary Fabyan Windeatt'],['Biography & Autobiography'],Tan Books & Pub,The story for children 10 and up of St. Hyacin...,"[0.0, 0.0, 0.0, 0.18298260867595673, 0.0, 0.0,..."


In [50]:
dataset['topic_vectors'] = dataset['topic_vectors'].apply(lambda x: np.zeros(num_topics) if isinstance(x, float) and np.isnan(x) else x)

In [ ]:
# dataset.insert(0,'bookID',dataset.index)
# dataset.to_pickle('dataset_final.pkl')

In [2]:
dataset = pd.read_pickle('/home/saji/DSprojects/BookRecommendation/data/dataset_final.pkl')

In [5]:
dataset.tail()

,bookID,Title,image,authors,categories,publisher,description,topic_vectors
143500,143500,The Orphan Of Ellis Island (Time Travel Advent...,http://books.google.com/books/content?id=J7M-N...,['Elvira Woodruff'],['Juvenile Fiction'],Scholastic Paperbacks,"During a school trip to Ellis Island, Dominick...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
143501,143501,Red Boots for Christmas,http://books.google.com/books/content?id=3n8k6...,NaN,['Juvenile Fiction'],NaN,Everyone in the village of Friedensdorf is hap...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
143502,143502,Mamaw,NaN,['Wild Wild Cabbage'],NaN,NaN,"Give your Mamaw a useful, beautiful and though...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
143503,143503,The Autograph Man,http://books.google.com/books/content?id=JM6YV...,['Zadie Smith'],['Fiction'],Vintage,Alex-Li Tandem sells autographs. His business ...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
143504,143504,Student's Solutions Manual for Johnson/Mowry's...,http://books.google.com/books/content?id=dehfP...,"['David B. Johnson', 'Thomas A. Mowry']",NaN,Brooks/Cole Publishing Company,Discover the many ways mathematics is relevant...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [3]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Assuming 'books' is your DataFrame with columns: 'Title', 'authors', 'categories', 'description', 'topic_vectors'
def show_image(val):
    return '<a href="{}"><img src="{}" width=50></img></a>'.format(val, val)
def calculate_similarity(vec1, vec2):
    # Helper function to calculate cosine similarity between two vectors
    return cosine_similarity([vec1], [vec2])[0][0]

def recommend_books(user_title, dataset):
    # Find the book in the dataset based on the provided title
    target_book = dataset.loc[dataset['Title'] == user_title].iloc[0]
    target_topic_dist = target_book['topic_vectors']
    target_author = target_book['authors']
    target_category = target_book['categories']

    # Calculate similarity scores using vectorized operations
    dataset['topic_similarity'] = dataset['topic_vectors'].apply(lambda x: calculate_similarity(target_topic_dist, x))
    dataset['author_similarity'] = np.where(dataset['authors'] == target_author, 1.0, 0.0)
    dataset['category_similarity'] = np.where(dataset['categories'] == target_category, 1.0, 0.0)

    # Combine the similarity scores into an overall score
    dataset['overall_similarity'] = (
        0.5 * dataset['topic_similarity'] +
        0.3 * dataset['author_similarity'] +
        0.2 * dataset['category_similarity']
    )

    # Sort books by similarity score and return the top N recommendations
    recommended_books = dataset.sort_values(by='overall_similarity', ascending=False)[:30]
    
    # If you want to return a subset of columns
    recommended_books = recommended_books[['bookID','Title', 'image', 'authors', 'categories', 'description', 'overall_similarity']]
    
    return recommended_books.reset_index(drop = True)

In [5]:
# Example usage:
top_n=7
user_input_title = "In Search of the Black Rose (Nancy Drew)"
recommended_books = recommend_books(user_input_title, dataset)
recommended_books[0:7]
recommended_books.head(top_n).style.format({'image':show_image})


,bookID,Title,image,authors,categories,description,overall_similarity
0,108438,In Search of the Black Rose (Nancy Drew),,['Carolyn Keene'],['Juvenile Fiction'],"A dinner party at England’s Oxford University is interrupted by an arrow shot that sends Nancy on a strange journey through the school’s medieval corridors. She must find out the meaning of a clue left behind, a black rose. The search for answers moves Nancy to explore the life—and death—of Dame Gwyneth Davies, the famous mystery writer and playwright. And as Nancy draws back a veil of family secrets, she uncovers a real-life drama that could end in tragedy.",1.000000
1,30826,Candy Is Dandy (Nancy Drew Notebooks No. 38),,['Carolyn Keene'],['Juvenile Fiction'],"SUGAR, COCOA, CARAMEL, CHOCOLATE -- A MISSING CANDY FORMULA MAKES A STICKY MESS! What could be sweeter than a visit to Zuckerman's Zonked Candy Factory? Nancy and her class will see how their favorite candy is made, and even get free samples. Mr. Zuckerman himself gives the grand tour past the sparkling machines that spit out Sticky Gummy Goo, Karamel Krunchettes, Frostee Jewels, and more. Then Mr. Zuckerman tells the kids he has dreamed up the perfect candy. He even shows them the paper with the secret recipe. But when the visit is over, the recipe has disappeared and no one can find it! Is the greatest candy lost for good?",0.948181
2,89203,Where's Nancy? (Nancy Drew: Girl Detective Super Mystery #1),,['Carolyn Keene'],['Juvenile Fiction'],"Nancy has been missing for several days, so her good friends George and Bess follow a slim trail of clues to try and track her down.",0.855246
3,67789,Jealous Feelings (Nancy Drew on Campus #20),,['Carolyn Keene'],['Juvenile Fiction'],"Her relationship with Jake strained when she dates Terry, Nancy wonders what is really going on with her friend George, while hospital volunteer Ginny receives suspicious attentions from handsome doctor Malcolm.",0.820106
4,80473,"The Clue in the Diary (Nancy Drew, Book 7)",,['Carolyn Keene'],['Juvenile Fiction'],"A special treat for Nancy Drew fans, and any reader who's new to the series! This is a stunning new edition of an old favourite,The Clue in the Diary, the seventh book in the incredibly popular, long-running series. It's the same exciting mystery that readers have fallen in love with for more than 80 years-Nancy must figure out the connection between a mysterious diary and a suspicious house fire. Now with a brand-new look, this is an edition that collectors won't want to miss!",0.803013
5,132583,The Doll House Mystery,,['Carolyn Keene'],['Juvenile Fiction'],"Nancy is intrigued with the beautiful antique dollhouse at Mrs. Rutledge's house, where she has volunteered to walk the woman's dog for volunteer week, but when some of the pieces turn up missing, Nancy becomes the prime suspect.",0.755504
6,81560,The Clue of the Black Keys (Nancy Drew Mystery Stories),,['Carolyn Keene'],['Mystery & Detective Stories'],Once again Nancy Drew is involved in a mystery as she tries to locate an archaeologist who disappeared with the clues to a buried treasure.,0.698362


In [17]:
user_input_title = "The Origin of Species"
recommended_books = recommend_books(user_input_title, dataset)
recommended_books.head(top_n).style.format({'image':show_image})


,bookID,Title,image,authors,categories,description,overall_similarity
49356,49356,The Origin of Species,,['Charles Darwin'],['Science'],"On the Origin of Species, published on 24 November 1859, is a work of scientific literature by Charles Darwin which is considered to be the foundation of evolutionary biology. Darwin's book introduced the scientific theory that populations evolve over the course of generations through a process of natural selection.",1.000000
45163,45163,On Evolution,,['Charles Darwin'],['Science'],"The father of evolution imbibed history with his controversial book On the Origin of Species. As a young scientist studying natural history, Charles Darwin set out on the HMS Beagle for a five-year voyage that would bring him in contact with animal species, plant species, and rock formations all over the world. This voyage established him as an eminent geologist, and his publication of his journal of the voyage established him as a well-known popular author. After continued, careful studying of his findings from his trip around the world, he published one of the most controversial books of history—The Origin of Species—which opposed the church’s theory that all species had come into existence at the time of the Creation. Darwin on Evolution contains quotes and wisdom from The Origin of Species, as well as from his other publications, pertaining to his theories of evolution and natural selection. Readers young and old alike will have the opportunity to learn more about his theories while also taking away important life lessons. As Darwin says, “In the long history of humankind (and animal kind, too) those who learned to collaborate and improvise most effectively have prevailed.”",0.925365
81820,81820,Conquest of Earth,,['Edward O. Wilson'],['Science'],"New York Times Bestseller From the most celebrated heir to Darwin comes a groundbreaking book on evolution, the summa work of Edward O. Wilson's legendary career. Sparking vigorous debate in the sciences, The Social Conquest of Earth upends “the famous theory that evolution naturally encourages creatures to put family first” (Discover). Refashioning the story of human evolution, Wilson draws on his remarkable knowledge of biology and social behavior to demonstrate that group selection, not kin selection, is the premier driving force of human evolution. In a work that James D. Watson calls “a monumental exploration of the biological origins of the human condition,” Wilson explains how our innate drive to belong to a group is both a “great blessing and a terrible curse” (Smithsonian). Demonstrating that the sources of morality, religion, and the creative arts are fundamentally biological in nature, the renowned Harvard University biologist presents us with the clearest explanation ever produced as to the origin of the human condition and why it resulted in our domination of the Earth’s biosphere.",0.674175
88283,88283,Conducting Research Surveys Via E-Mail and The Web,,"['Matthias Schonlau', 'Ronald D., Jr. Fricker', 'Marc N. Elliott']",['Science'],"Internet-based surveys, although still in their infancy, are becomingincreasingly popular because they are believed to be faster, better,cheaper, and easier to conduct than surveys using more-traditional telephoneor mail methods. Based on evidence in the literature and real-life casestudies, this book examines the validity of those claims. The authorsdiscuss the advantages and disadvantages of using e-mail and the Web toconduct research surveys, and also offer practical suggestions for designing and implementing Internet surveys most effectively.Among other findings, the authors determined that Internet surveys may bepreferable to mail or telephone surveys when a list of e-mail addresses forthe target population is available, thus eliminating the need for mail orphone invitations to potential respondents. Internet surveys also arewell-suited for larger survey efforts and for some target populations thatare difficult to reac

In [18]:
user_input_title = "The Hungry Ghosts"
recommended_books = recommend_books(user_input_title, dataset)
recommended_books.head(top_n).style.format({'image':show_image})


,bookID,Title,image,authors,categories,description,overall_similarity
42012,42012,The Hungry Ghosts,,"['Gabor Maté, MD']",['Self-Help'],"From bestselling author Gabor Maté, the essential resource for understanding the roots and behaviors of addiction--now with an added introduction by the author. Based on Gabor Maté’s two decades of experience as a medical doctor and his groundbreaking work with the severely addicted on Vancouver’s skid row, In the Realm of Hungry Ghosts radically reenvisions this much misunderstood field by taking a holistic approach. Dr. Maté presents addiction not as a discrete phenomenon confined to an unfortunate or weak-willed few, but as a continuum that runs throughout (and perhaps underpins) our society; not a medical ""condition"" distinct from the lives it affects, rather the result of a complex interplay among personal history, emotional, and neurological development, brain chemistry, and the drugs (and behaviors) of addiction. Simplifying a wide array of brain and addiction research findings from around the globe, the book avoids glib self-help remedies, instead promoting a thorough and compassionate self-understanding as the first key to healing and wellness. In the Realm of Hungry Ghosts argues persuasively against contemporary health, social, and criminal justice policies toward addiction and those impacted by it. The mix of personal stories—including the author’s candid discussion of his own ""high-status"" addictive tendencies—and science with positive solutions makes the book equally useful for lay readers and professionals.",1.000000
73210,73210,Burn This Book ... and Move On with Your Life,,['Jessica Hurley'],['Self-Help'],"Most people carry around emotional baggage without realizing how it negatively affects the quality of their everyday lives. Burn This Book . . . and Move On with Your Life helps readers identify those issues that are dragging them down, whether they are fear or pride or anger, and then gives them a way to purge their problems from their lives. Each page of Burn This Book addresses a new topic, such as ""I will no longer blame my parents for my choices"" or ""I will let go of the need to be perfect,"" and includes supporting text that examines that particular issue.. Once the reader has contemplated a particular problem, he or she tears out that perforated page, internalizes the simple lesson, then casts it away for good by carefully putting it to flame. Burn This Book offers a uniquely active and involved reading experience - one in which symbolic destruction through fire brings mental rejuvenation. It's an ideal way for readers to free themselves from restrictive thinking by taking dramatic action. From cover to cover, Burn This Book . . . and Move On with Your Life is a refreshing way to resolve problems and make life better.",0.670056
12685,12685,Up From Here: Reclaiming the Male Spirit: A Guide to Transforming Emotions into Power and Freedom,,['Iyanla Vanzant'],['Self-Help'],"For men — and the women who love them — a guide to transformation and true power The world needs your light. The world needs your life. Every woman and child needs you also. Few men are taught the true spiritual essence of masculinity. In Up from Here Iyanla Vanzant empowers men with the insight and skills they need to identify, name, and understand their feelings, as well as to overcome the old patterns of behavior that result from misusing or not tapping into their emotional strength. In clear terms Vanzant shows men how to use the energy of emotions to move beyond painful, negative feelings such as rage, shame, fear, and anger to authenticity and strength. Like the phoenix, you can rise up from here, out of the ashes of the crash and burn of the past, to claim your true identity and power. By using Vanzant's ""power tools"" men can transform their frustrations and failures in love, life, and work into opportunities to claim their true potential and purpose.",0.649482
120899,120899,The John W

In [19]:
user_input_title = "Harry Potter and the Chamber of Secrets"
recommended_books = recommend_books(user_input_title, dataset)
recommended_books.head(top_n).style.format({'image':show_image})


,bookID,Title,image,authors,categories,description,overall_similarity
46162,46162,Harry Potter and the Chamber of Secrets,,['J. K. Rowling'],['Juvenile Fiction'],"Witchcraft, wizardry - fiction.",1.000000
27406,27406,Harry Potter and The Sorcerer's Stone,,['J. K. Rowling'],['Juvenile Fiction'],"Celebrate 20 years of Harry Potter magic! Harry Potter has never even heard of Hogwarts when the letters start dropping on the doormat at number four, Privet Drive. Addressed in green ink on yellowish parchment with a purple seal, they are swiftly confiscated by his grisly aunt and uncle. Then, on Harry's eleventh birthday, a great beetle-eyed giant of a man called Rubeus Hagrid bursts in with some astonishing news: Harry Potter is a wizard, and he has a place at Hogwarts School of Witchcraft and Wizardry. An incredible adventure is about to begin!These new editions of the classic and internationally bestselling, multi-award-winning series feature instantly pick-up-able new jackets by Jonny Duddle, with huge child appeal, to bring Harry Potter to the next generation of readers. It's time to PASS THE MAGIC ON ...",0.691871
119837,119837,Phantom Stallion #16: The Wildest Heart,,['Terri Farley'],['Juvenile Fiction'],"The further adventures in the wildly popular series about Samantha and her mustang the Phantom. In Firefly, the Phantom's wild colt has been injured, and Sam must help him find a new home.",0.671146
100876,100876,Bambi,,"['Felix Salten', 'Richard Cowdrey']",['Juvenile Fiction'],The classical tale of a young fawn's growing maturity and independence as he learns to face the hardships of his life.,0.664246
121323,121323,Odd Jobs: The Wackiest Jobs You've Never Heard Of,,['Elenor Fremont'],['Juvenile Fiction'],Have you ever wondered... Who would wrangle roaches for a living? What in the world is a food stylist? And just how does a person become a professional armpit-sniffer? This book will show you all the strangest jobs around -- and introduce you to the real people who get paid to do them! Professional Funnyman Gator Aide Master Blaster Storm Chaser Professional Scary Guy Feather Guy Sound Effects Whiz Food Stylist Roller Coaster Designer Master Sniffer Muppet Exploder Bug Wrangler and Animal Handler,0.662471
48951,48951,Little Red Riding Hood,,['Paul Galdone'],['Juvenile Fiction'],A retelling of the folk tale about a little girl who finds a wolf in her grandmother's clothing.,0.662068
67663,67663,Fossil,,nan,['Juvenile Fiction'],"When a boy discovers that breaking open rocks with fossils brings the fossils back to life, he ends up bringing something back that is bigger than he can handle.",0.657107


In [20]:
top_n = 7
user_input_title = "Stories of English"
recommended_books = recommend_books(user_input_title, dataset)
recommended_books.head(top_n).style.format({'image':show_image})


,bookID,Title,image,authors,categories,description,overall_similarity
106280,106280,Stories of English,,['David Crystal'],['History'],"The English language is now accepted as the global lingua franca of the modern age, spoken or written in by over a quarter of the human race. But how did it evolve? How did a language spoken originally by a few thousand Anglo-Saxons become one used by more than 1,500 million? What developments can be seen as we move from Beowulf to Chaucer to Shakespeare to Dickens and the present day? A host of fascinating questions are answered in The Stories of English, a groundbreaking history of the language by David Crystal, the world-renowned writer and commentator on English. Many books have been written about English, but they have all focused on a single variety: the educated, printed language called “standard†? English. David Crystal turns the history of English on its head and instead provides a startlingly original view of where the richness, creativity and diversity of the language truly lies—in the accents and dialects of nonstandard English users all over the world. Whatever their regional, social or ethnic background, each group has a story worth telling, whether it is in Scotland or Somerset, South Africa or Singapore. Interweaved within this central chronological story are accounts of uses of dialect around the world as well as in literary classics from The Canterbury Tales to The Lord of the Rings. For the first time, regional speech and writing is placed center stage, giving a sense of the social realities behind the development of English. This significant shift in perspective enables the reader to understand for the first time the importance of everyday, previously marginalized, voices in our language and provides an argument too for the way English should be taught in the future.",1.000000
99624,99624,Priestley: Political Writings (Cambridge Texts in the History of Political Thought),,"['Joseph Priestley', 'Priestley Joseph']",['History'],"Joseph Priestley (1733-1804) was arguably one of the most important and interesting English theorists to focus on the issue of political liberty during the English Enlightenment. His concept of freedom is of crucial importance to two of the major issues of his day: the right of dissenters to religious toleration, and the right of the American colonists to self-government. Despite the fundamental importance of both these themes in liberal political theory and their contemporary relevance to national self-determination, Priestley's writings lack a modern edition. This new collection will be the first to make accessible to students Priestleys' Essay on the First Principles and The Present State of Liberty, which encapsulate his political ideology. An introduction and notes, together with guides to further reading and key figures in the text provide the student with all the material necessary for approaching Priestley.",0.673546
54940,54940,Separation and Its Discontents: Toward an Evolutionary Theory of Anti-Semitism,,['Kevin MacDonald'],['History'],"The Lost 13th Tribe: America in Prophecy by Dr. Marlin Kime is the first volume of a multi-volume work that, for the first time, answers the most important questions about America. Who are we in history? Where did we come from? What does the future hold for us? For the first century and a half of America's existence, it was invincible as it achieved unprecedented heights of power, wealth, and greatness. And it did all of this in the face of daunting, hopeless and impossible obstacles arrayed against it. These volumes show exactly how and why this was accomplished. Conversely, why was America able to achieve such overwhelming and incredible victories against its powerful Axis foes of World War II and yet fail miserably against the third rate Third World nations of Korea and Vietnam? These volumes thoroughly explain the hidden reasons behind these enigmas of history. Centuries ago, all of this was foretold in t

In [21]:
user_input_title = "A Kiss Is Just A Kiss"
recommended_books = recommend_books(user_input_title, dataset)
recommended_books.head(top_n).style.format({'image':show_image})

,bookID,Title,image,authors,categories,description,overall_similarity
107859,107859,A Kiss Is Just A Kiss,,['Denise Hunter'],['Fiction'],"Riley Callahan has plans to finally reveal his secret feelings for his best friend (who just happens to be his brother’s ex). But tragedy strikes and derails his plans. Watching the love of his life fall for his brother was enough to send Riley straight to boot camp. But over a year later, he’s officially a marine, and Beau and Paige are no longer an item. When Riley’s tour in Afghanistan is up, he intends to confess his feelings to Paige and win his best friend’s heart once and for all. But all that changes when an IED takes the life of a comrade and leaves Riley an amputee. Now he’s heading home, injured and troubled. His plans to win Paige are a distant dream. She deserves so much more than the man that’s left. All he can do now is put some healthy distance between them. But upon his return he discovers his family has arranged for him to stay with Paige. Paige is a nurturer at heart and happy to take care of her best buddy. By all appearances Riley is adjusting miraculously well to his disability. But as the days pass, Paige begins to see that the smiles and laughter are just a mask for the pain he’s hiding. To make matters worse, her job is in serious jeopardy. The animal shelter that she’s poured her heart into has lost its funding, and she has three months to come up with the money needed to save it. As the weeks wear on, Paige’s feelings for Riley begin to shift into uncharted territory. Why is she suddenly noticing his arm muscles and the way his lips curl at the corners? Will she be able to deny her feelings for another Callahan brother? And will Riley let his heart heal so he can let Paige in? Full-length Contemporary Romance Includes Discussion Questions for Book Clubs Part of the Summer Harbor series Book One: Falling Like Snowflakes Book Two: The Goodbye Bride Book Three: Just a Kiss",1.000000
113082,113082,MILWAUKEE AUTUMNS CAN BE LETHAL,,['Kathleen Anne Barrett'],['Fiction'],"Legal researcher and occasional sleuth Beth Hartley must track a killer when Don Balstrum, the man she is working for, is brutally murdered, plunging her a world of corruption and greed where she is confronted by a wealth of suspects. Reprint.",0.686212
24783,24783,"Dead men's money,",,['J. S. Fletcher'],['Fiction'],"J.S. Fletcher was one of the most popular detective fiction writers during the ""Golden Era"" that is now most associated with the likes of Sherlock Holmes and Arthur Conan Doyle. Fletcher wrote over 200 books during his career.",0.683891
9101,9101,The Man in the Black Suit: 4 Dark Tales,,['Stephen King'],['Fiction'],"A new collection of short fiction features ""L.T.'s Theory of Pets,"" ""Lunch at the Gotham Cafâe,"" ""Riding the Bullet,"" ""1408,"" and ""In the Deathroom.""",0.679505
15121,15121,The Elves: Book 1 of the E.F.H. Trilogy,,['Margaret R. Gifford'],['Fiction'],"In The Elves: Book 1 of the E.F. H. Trilogy, we meet Rachel, a young princess from Outworld who's thrown into a warring world called Inworld. Taken along on a journey filled with dark elves, fire sprites, tree sprites, and fairies; Rachel befriends both sides of the war. Morak, a young light elf, takes responsibility for Rachel and worries about the violent nightmares that plague her sleep. Visions of a two-headed dragon, strange symbols, and a woman who looks like her dead mother haunt Rachel's dreams. She finds herself in the high elfin council, where she learns her presence in Inworld is a crime punishable by death. She tries to convince the elves that their war is pointless, but no one listens. They don't see that while they fight, an even greater evil is working itself through their world, and can spread into Outworld too. Is Rachel the key to it all?",0.674859
45918,45918,The Last Great Dance on Earth,,['Sandra Gulland'],['Fiction'],"The Last Great Dance on Earth is the triumphant final volume of Sandra Gulland's belo

In [22]:
user_input_title = "Mein Kampf: My Struggle"
recommended_books = recommend_books(user_input_title, dataset)
recommended_books.head(top_n).style.format({'image':show_image})

,bookID,Title,image,authors,categories,description,overall_similarity
126065,126065,Mein Kampf: My Struggle,,['Adolf Hitler'],['Biography & Autobiography'],"‘MEIN KAMPF’ is the autobiography of Adolf Hitler gives detailed insight into the mission and vision of Adolf Hitler that shook the world. This book is the merger of two volumes. The first volume of MEIN KAMPF’ was written while the author was imprisioned in a Bavarian fortress. The book deals with events which brought the author into this blight. It was the hour of Germany’s deepest humiliation, when Napolean has dismembered the old German Empire and French soldiers occupied almost the whole of Germony. The books narrates how Hitler was arrested with several of his comrades and imprisoned in the fortress of Landsberg on the river Lech. During this period only the author wrote the first volume of MEIN KAMPF. The Second volume of MEIN KAMPF was written after release of Hitler from prison and it was published after the French had left the Ruhr, the tramp of the invading armies still echoed in German ears and the terrible ravages had plunged the country into a state of social and economic Chaos. The beauty of the book is, MEIN KAMPF is an historical document which bears the emprint of its own time. Moreover, Hitler has declared that his acts and ‘public statements’ constitute a partial revision of his book and are to be taken as such. Also, the author has translated Hitler’s ideal, the Volkischer Staat, as the People’s State. The author has tried his best making German Vocabulary easy to understand. You will never be satisfied until go through the whole book. A must read book, which is one of the most widely circulated and read books worldwide.",1.000000
47202,47202,Blame Hitler,,['Adolf Hitler'],['Biography & Autobiography'],"‘MEIN KAMPF’ is the autobiography of Adolf Hitler gives detailed insight into the mission and vision of Adolf Hitler that shook the world. This book is the merger of two volumes. The first volume of MEIN KAMPF’ was written while the author was imprisioned in a Bavarian fortress. The book deals with events which brought the author into this blight. It was the hour of Germany’s deepest humiliation, when Napolean has dismembered the old German Empire and French soldiers occupied almost the whole of Germony. The books narrates how Hitler was arrested with several of his comrades and imprisoned in the fortress of Landsberg on the river Lech. During this period only the author wrote the first volume of MEIN KAMPF. The Second volume of MEIN KAMPF was written after release of Hitler from prison and it was published after the French had left the Ruhr, the tramp of the invading armies still echoed in German ears and the terrible ravages had plunged the country into a state of social and economic Chaos. The beauty of the book is, MEIN KAMPF is an historical document which bears the emprint of its own time. Moreover, Hitler has declared that his acts and ‘public statements’ constitute a partial revision of his book and are to be taken as such. Also, the author has translated Hitler’s ideal, the Volkischer Staat, as the People’s State. The author has tried his best making German Vocabulary easy to understand. You will never be satisfied until go through the whole book. A must read book, which is one of the most widely circulated and read books worldwide.",0.855539
29547,29547,How to draw animals,,['Jack Hamm'],['Biography & Autobiography'],"Simple, clear instructions for drawing animals with more than a thousand step-by-step illustrations. Basic fundamentals for the beginner, new principles and techniques for the professional. A detailed guide for everyone who enjoys—or wants to enjoy—drawing.",0.677431
116597,116597,Pride of Family : Four Generations of American Women of Color,,['Carole Ione'],['Biography & Autobiography'],"“From the moment I read the words [my great-grandmother] Frances Anne Rollin wrote in Boston on January 1, 1868—“The year renews its birth toda

////////////////////////////////

In [ ]:
# dataset['image'] = dataset['image'].fillna('https://www.shutterstock.com/image-illustration/black-solid-dark-night-260nw-1939034020.jpg')
# dataset = dataset.drop(columns=['publisher'],axis=1)
# dataset = dataset.dropna()
#books.to_pickle('dataset.pkl')